In [41]:
import MetaTrader5 as mt5

from atj_trading.backtester import get_ohlc_history, Backtester
from datetime import datetime, timedelta, time

import plotly.express as px 

In [42]:
mt5.initialize()

symbol = 'ITALY40'
start_dt = datetime.now() - timedelta(days=365*2)
end_dt = datetime.now()

ohlc_df = get_ohlc_history(symbol, mt5.TIMEFRAME_M15, start_dt, end_dt)
ohlc_df

,time,open,high,low,close
0,2022-09-05 09:00:00,21269.7,21374.7,21237.2,21362.2
1,2022-09-05 09:15:00,21364.7,21409.7,21364.7,21387.1
2,2022-09-05 09:30:00,21387.1,21409.6,21387.1,21392.1
3,2022-09-05 09:45:00,21392.1,21444.6,21372.0,21417.0
4,2022-09-05 10:00:00,21409.5,21454.5,21387.0,21389.5
...,...,...,...,...,...
28454,2024-09-02 18:00:00,34328.3,34345.4,34320.9,34345.4
28455,2024-09-02 18:15:00,34345.4,34345.4,34320.5,34335.5
28456,2024-09-02 18:30:00,34330.5,34340.5,34310.6,34333.0
28457,2024-09-02 18:45:00,34333.0,34335.6,34328.1,34330.6


In [43]:
ohlc_df['gain'] = ohlc_df['open'].shift(-1) - ohlc_df['open'] 
ohlc_df['perc_gain'] = (ohlc_df['gain'] / ohlc_df['open']) * 100
ohlc_df['time_hms'] = ohlc_df['time'].dt.time
ohlc_df['year'] = ohlc_df['time'].dt.year
ohlc_df['month'] = ohlc_df['time'].dt.month
ohlc_df['day_of_week'] = ohlc_df['time'].dt.dayofweek

ohlc_df = ohlc_df[ohlc_df['time_hms'].between(time(2, 0, 0), time(23, 0, 0))]

ohlc_df

,time,open,high,low,close,gain,perc_gain,time_hms,year,month,day_of_week
0,2022-09-05 09:00:00,21269.7,21374.7,21237.2,21362.2,95.0,0.446645,09:00:00,2022,9,0
1,2022-09-05 09:15:00,21364.7,21409.7,21364.7,21387.1,22.4,0.104846,09:15:00,2022,9,0
2,2022-09-05 09:30:00,21387.1,21409.6,21387.1,21392.1,5.0,0.023379,09:30:00,2022,9,0
3,2022-09-05 09:45:00,21392.1,21444.6,21372.0,21417.0,17.4,0.081338,09:45:00,2022,9,0
4,2022-09-05 10:00:00,21409.5,21454.5,21387.0,21389.5,-17.5,-0.081739,10:00:00,2022,9,0
...,...,...,...,...,...,...,...,...,...,...,...
28454,2024-09-02 18:00:00,34328.3,34345.4,34320.9,34345.4,17.1,0.049813,18:00:00,2024,9,0
28455,2024-09-02 18:15:00,34345.4,34345.4,34320.5,34335.5,-14.9,-0.043383,18:15:00,2024,9,0
28456,2024-09-02 18:30:00,34330.5,34340.5,34310.6,34333.0,2.5,0.007282,18:30:00,2024,9,0
28457,2024-09-02 18:45:00,34333.0,34335.6,34328.1,34330.6,-4.9,-0.014272,18:45:00,2024,9,0


In [44]:
ohlc_df_by_hour = ohlc_df.groupby(['month', 'time_hms'], as_index=False)['perc_gain'].mean()
ohlc_df_by_hour

,month,time_hms,perc_gain
0,1,09:00:00,0.027261
1,1,09:15:00,0.020215
2,1,09:30:00,-0.013417
3,1,09:45:00,0.027649
4,1,10:00:00,0.029712
...,...,...,...
670,12,21:45:00,0.007621
671,12,22:00:00,0.007993
672,12,22:15:00,-0.001238
673,12,22:30:00,0.004688


In [45]:
fig = px.bar(ohlc_df_by_hour, x='time_hms', y='perc_gain', color='month')
fig

In [46]:
ohlc_df_by_hour_day = ohlc_df.groupby(['day_of_week', 'time_hms'], as_index=False)['perc_gain'].mean()
ohlc_df_by_hour_day

,day_of_week,time_hms,perc_gain
0,0,09:00:00,0.017664
1,0,09:15:00,0.022025
2,0,09:30:00,0.002865
3,0,09:45:00,0.022623
4,0,10:00:00,0.036702
...,...,...,...
280,4,22:00:00,0.008724
281,4,22:15:00,0.005804
282,4,22:30:00,-0.003333
283,4,22:45:00,-0.057230


In [47]:
fig = px.bar(ohlc_df_by_hour_day, x='time_hms', y='perc_gain', color='day_of_week')
fig

In [48]:
# create trade logic
def on_bar(data, trades, orders):
    
    open_trades = trades[trades['state'] == 'open']
    num_open_trades = open_trades.shape[0]
    
    # entry signal
    if data['time_hms'] == time(10, 45, 0) and data['day_of_week'] in [3, 4]:
        print(data['time'], 'selling')
        orders.open_trade('NZDJPY', 100000, 'sell')
    
    elif data['time_hms'] == time(15, 30, 0) and data['day_of_week'] in [3, 4]:
        print(data['time'], 'buying')
        orders.open_trade('NZDJPY', 100000, 'buy')
        
    # exit signal
    if num_open_trades:
        trade = open_trades.iloc[0]

        if trade['order_type'] == 'sell' and data['time_hms'] == time(15, 30, 0):
            orders.close_trade(trade)
        elif trade['order_type'] == 'buy' and data['time_hms'] == time(15, 45, 0):
            orders.close_trade(trade)

In [49]:
# backtest parameters
starting_balance = 100000
currency = 'EUR'
exchange_rate = 1/150
commission = -7 / 100000 

# backtest
bt = Backtester()
bt.set_starting_balance(starting_balance, currency=currency)
bt.set_exchange_rate(exchange_rate)
bt.set_commission(commission)

bt.set_historical_data(ohlc_df)
bt.set_on_bar(on_bar)

bt.run_backtest()

bt.trades

2022-09-08 15:15:00 selling
2022-09-08 15:30:00 buying
2022-09-09 15:15:00 selling
2022-09-09 15:30:00 buying
2022-09-15 15:15:00 selling
2022-09-15 15:30:00 buying
2022-09-16 15:15:00 selling
2022-09-16 15:30:00 buying
2022-09-22 15:15:00 selling
2022-09-22 15:30:00 buying
2022-09-23 15:15:00 selling
2022-09-23 15:30:00 buying
2022-09-29 15:15:00 selling
2022-09-29 15:30:00 buying
2022-09-30 15:15:00 selling
2022-09-30 15:30:00 buying
2022-10-06 15:15:00 selling
2022-10-06 15:30:00 buying
2022-10-07 15:15:00 selling
2022-10-07 15:30:00 buying
2022-10-13 15:15:00 selling
2022-10-13 15:30:00 buying
2022-10-14 15:15:00 selling
2022-10-14 15:30:00 buying
2022-10-20 15:15:00 selling
2022-10-20 15:30:00 buying
2022-10-21 15:15:00 selling
2022-10-21 15:30:00 buying
2022-10-27 15:15:00 selling
2022-10-27 15:30:00 buying
2022-10-28 15:15:00 selling
2022-10-28 15:30:00 buying
2022-11-03 15:15:00 selling
2022-11-03 15:30:00 buying
2022-11-04 15:15:00 selling
2022-11-04 15:30:00 buying
2022-11-10

,state,symbol,order_type,volume,open_time,open_price,close_time,close_price,sl,tp,info,profit,commission,profit_net,profit_cumulative,balance
0,closed,NZDJPY,sell,100000,2022-09-08 15:15:00,21445.5,2022-09-08 15:30:00,21454.8,0,0,None,-6200.00,-7.0,-6207.0,-6207.0,93793.0
1,closed,NZDJPY,buy,100000,2022-09-08 15:30:00,21454.8,2022-09-08 15:45:00,21410.1,0,0,None,-29800.00,-7.0,-29807.0,-36014.0,63986.0
2,closed,NZDJPY,sell,100000,2022-09-09 15:15:00,22075.4,2022-09-09 15:30:00,22069.9,0,0,None,3666.67,-7.0,3659.67,-32354.33,67645.67
3,closed,NZDJPY,buy,100000,2022-09-09 15:30:00,22069.9,2022-09-09 15:45:00,22148.7,0,0,None,52533.33,-7.0,52526.33,20172.0,120172.0
4,closed,NZDJPY,sell,100000,2022-09-15 15:15:00,22483.3,2022-09-15 15:30:00,22402.2,0,0,None,54066.67,-7.0,54059.67,74231.67,174231.67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
405,closed,NZDJPY,buy,100000,2024-08-23 15:30:00,33578.3,2024-08-23 15:45:00,33612.4,0,0,None,22733.33,-7.0,22726.33,2491.31,102491.31
406,closed,NZDJPY,sell,100000,2024-08-29 15:15:00,34150.6,2024-08-29 15:30:00,34151.1,0,0,None,-333.33,-7.0,-340.33,2150.98,102150.98
407,closed,NZDJPY,buy,100000,2024-08-29 15:30:00,34151.1,2024-08-29 15:45:00,34115.3,0,0,None,-23866.67,-7.0,-23873.67,-21722.69,78277.31
408,closed,NZDJPY,sell,100000,2024-08-30 15:15:00,34410.4,2024-08-30 15:30:00,34430.1,0,0,None,-13133.33,-7.0,-13140.33,-34863.02,65136.98


In [50]:
pnl_chart = bt.plot_pnl()
pnl_chart